# Kissipo-DNN-Mnist(1)
MNIST 手寫數字辨識
![Mnist](https://camo.githubusercontent.com/01c057a753e92a9bc70b8c45d62b295431851c09cffadf53106fc0aea7e2843f/687474703a2f2f692e7974696d672e636f6d2f76692f3051493378675875422d512f687164656661756c742e6a7067)

##  載入TF2.0所需的函數庫


In [ ]:
import tensorflow as tf
print(tf.__version__)

# Part-1 The training phase

## IPO-M: Building a DL model (Sequential API, Graph mode)
$M(x)=\hat{y}$

In [ ]:
#Sequential API
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

In [ ]:
#Functional API
inputs = tf.keras.Input(shape=(784,))
L2_outputs = tf.keras.layers.Dense(128, activation="relu")(inputs)
outputs = tf.keras.layers.Dense(10)(L2_outputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

## IPO-M: Building a DL model (Subclassing API, Eager mode)
$M(x)=\hat{y}$

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10)

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

## IPO-I: input data

### 載入MNIST數據集。

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
type(x_train)

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [13]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## IPO-O: ontput data

In [ ]:
print(y_train.shape)
print(y_test.shape)

## IPO-P: Training model

###  選擇一個優化器和損失函數進行模型建構
The from_logits=True attribute inform the loss function that the output values generated by the model are not normalized, a.k.a. logits. In other words, the softmax function has not been applied on them to produce a probability distribution.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
#Create an optimizer with the desired parameters.
opt = tf.keras.optimizers.SGD(learning_rate=0.1)

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


## train_step

In [11]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

## test_step

In [12]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

### Compile model(Eager mode 不用編譯compile)

In [ ]:
#model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

##  進行模型訓練

In [ ]:
#model.fit(x_train, y_train, epochs=10)

In [14]:
EPOCHS = 5
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.2597472369670868, Accuracy: 92.47166442871094, Test Loss: 0.13441671431064606, Test Accuracy: 96.10000610351562
Epoch 2, Loss: 0.11587721109390259, Accuracy: 96.5250015258789, Test Loss: 0.10684952139854431, Test Accuracy: 96.83999633789062
Epoch 3, Loss: 0.08032161742448807, Accuracy: 97.47000122070312, Test Loss: 0.0971354991197586, Test Accuracy: 97.0999984741211
Epoch 4, Loss: 0.059014417231082916, Accuracy: 98.21166229248047, Test Loss: 0.08810698240995407, Test Accuracy: 97.3699951171875
Epoch 5, Loss: 0.04519445076584816, Accuracy: 98.61166381835938, Test Loss: 0.07974237948656082, Test Accuracy: 97.58999633789062


## IPO-P: Evaluating model (模型評估)

In [ ]:
model.evaluate(x_test,  y_test)

In [ ]:
import numpy as np
from PIL import Image
from IPython.display import display

# Part-2 The Inference phase

## IPO-M: Predicting the digit of an image
$M(x)=\hat{y}$

## IPO-I2: input a single image

In [ ]:
### 第一次

In [ ]:
#show image real size
img = Image.open('n1.bmp')
display(img)
img2=img.resize((28, 28))
img.close()

In [ ]:
#show image at the size of 28x28
display(img2)

### Covert an image to a numpy.ndarray (1, 28, 28)

In [ ]:
im2arr = np.array(img2)/ 255.0
im2arr = im2arr.reshape(1,28,28)

## IPO-P2: model.predict

In [ ]:
y_pred = model.predict(im2arr)
print(y_pred)


In [ ]:
y_hat =np.argmax(y_pred)
y_hat